# 2.2 - Selección de características

En este notebook voy a seleccionar las columnas importantes del dataset `listings`. Utilizaré tres metodos distintos. Por un lado la correlación, con los métodos de Pearson, Spearman y la Tau de Kendall, para intentar ver correlaciones entre las variables y el objetivo, y si existe colinealidad. 

Además usaré un OLS (Ordinary Least Squares - Mínimos Cuadrados Ordinarios), básicamente una regresión lineal, para determinar los p-values según el F-test de cada variable. 

También usaré un random forest o xgboost, no con el objetivo de predecir, sino para que me diga cuál es la importancia de las características.

In [1]:
# librerias

import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [2]:
listings=pd.read_csv('../data/transform_data/listings.csv')

listings=listings.drop(columns=['id', 'host_id'])

listings=listings[(listings.price>=10) & (listings.price=<196)]

for c in listings.select_dtypes(include='int'):
    listings[c]=pd.to_numeric(listings[c], downcast='integer')

for c in listings.select_dtypes(include='float'):
    listings[c]=pd.to_numeric(listings[c], downcast='float')
    
listings.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17968 entries, 0 to 21311
Columns: 240 entries, host_is_superhost to suitable_for_events
dtypes: float32(3), int16(10), int32(1), int8(224), object(2)
memory usage: 6.9 MB


### 1) Correlación

In [3]:
def correlacion(metodo: str, umbral: float) -> None:
    
    """
    Esta función calcula la correlación del dataframe 
    y muestra la columnas correlacionadas con el precio.
    
    param metodo: string, metodo de correlación (pearson, spearman, kendall)
    
    return: None (solo printea)
    """
    
    corr=listings._get_numeric_data().corr(method=metodo)

    display(corr.price[corr.price > umbral].sort_values(ascending=False)[1:])

    print('\n\033[1m' + 'Correlación negativa con el precio.' + '\033[0m')
    print(corr.price[corr.price < -umbral].sort_values(ascending=True))

In [4]:
correlacion('pearson', 0.1)

accommodates                                   0.575794
cleaning_fee                                   0.428574
air_conditioning                               0.378497
bedrooms                                       0.358680
beds                                           0.356993
guests_included                                0.355316
tv                                             0.323897
dishwasher                                     0.249624
crib                                           0.240532
hair_dryer                                     0.236680
family_kid_friendly                            0.234675
iron                                           0.228396
security_deposit                               0.223139
coffee_maker                                   0.214136
washer                                         0.201105
oven                                           0.182781
kitchen                                        0.176213
high_chair                                     0


Correlación negativa con el precio.
room_type_private_room                         -0.607842
calculated_host_listings_count_private_rooms   -0.236309
lock_on_bedroom_door                           -0.209718
free_street_parking                            -0.178777
room_type_shared_room                          -0.139754
smoking_allowed                                -0.129921
calculated_host_listings_count_shared_rooms    -0.127413
property_type_house                            -0.126582
pets_live_on_this_property                     -0.109517
y                                              -0.105206
Name: price, dtype: float64


Desde el punto de vista lineal de la $rho$ de Pearson, prácticamente no existe correlación con el precio. Veamos que es lo que ocurre con el punto de vista de Spearman, donde se busca una relación monótona. En una relación monótona, las variables tienden a cambiar al mismo tiempo, pero no necesariamente a un ritmo constante.

In [5]:
correlacion('spearman', 0.2)

accommodates                                   0.637747
calculated_host_listings_count_entire_homes    0.575809
beds                                           0.461002
cleaning_fee                                   0.459942
guests_included                                0.433070
air_conditioning                               0.403420
bedrooms                                       0.369407
security_deposit                               0.366275
tv                                             0.353078
hair_dryer                                     0.252541
family_kid_friendly                            0.245876
iron                                           0.240857
dishwasher                                     0.240512
crib                                           0.239420
coffee_maker                                   0.229122
washer                                         0.216259
Name: price, dtype: float64


Correlación negativa con el precio.
room_type_private_room                         -0.660642
calculated_host_listings_count_private_rooms   -0.597380
lock_on_bedroom_door                           -0.222941
Name: price, dtype: float64


In [6]:
correlacion('kendall', 0.2)

accommodates                                   0.502423
calculated_host_listings_count_entire_homes    0.431288
beds                                           0.362336
cleaning_fee                                   0.348891
guests_included                                0.346732
air_conditioning                               0.332910
tv                                             0.291366
bedrooms                                       0.289737
security_deposit                               0.285226
hair_dryer                                     0.208401
family_kid_friendly                            0.202902
Name: price, dtype: float64


Correlación negativa con el precio.
room_type_private_room                         -0.545174
calculated_host_listings_count_private_rooms   -0.456566
Name: price, dtype: float64


### 2) OLS

In [7]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [8]:
X=listings._get_numeric_data().drop('price', axis=1)

y=listings.price

In [9]:
modelo=sm.OLS(y, np.asarray(X)).fit()

pred=modelo.predict(X)

#modelo.summary()

In [10]:
p_values=modelo.summary().tables[1].as_html()

p_values=pd.read_html(p_values, header=0, index_col=0)

p_values=pd.DataFrame(p_values[0])

p_values['col']=X.columns.tolist()

In [13]:
p_values[['P>|t|', 'col']].sort_values(by='P>|t|')

,P>|t|,col
x1,0.000,host_is_superhost
x203,0.000,luggage_dropoff_allowed
x201,0.000,washer
x193,0.000,coffee_maker
x190,0.000,dishwasher
x185,0.000,patio_or_balcony
x182,0.000,elevator
x171,0.000,fireplace_guards
x160,0.000,bed_linens
x149,0.000,internet


In [14]:
p_values[p_values['P>|t|'] < 0.05]['col']

x1                                 host_is_superhost
x2                                      accommodates
x3                                         bathrooms
x4                                          bedrooms
x6                                  security_deposit
x7                                      cleaning_fee
x10                                   minimum_nights
x12                                  availability_30
x15                                 availability_365
x17                            number_of_reviews_ltm
x18                   calculated_host_listings_count
x19      calculated_host_listings_count_entire_homes
x20     calculated_host_listings_count_private_rooms
x21      calculated_host_listings_count_shared_rooms
x22                                                x
x23                                                y
x24                                                z
x54                             room_type_hotel_room
x55                           room_type_privat

### 3) Feature importances

In [15]:
from sklearn.ensemble import RandomForestRegressor as RFR

In [16]:
rfr=RFR().fit(X, y)

In [17]:
dict(zip(X.columns, rfr.feature_importances_))   

{'host_is_superhost': 0.002425504880140496,
 'accommodates': 0.023676373005488527,
 'bathrooms': 0.023195895433570413,
 'bedrooms': 0.06932862931750994,
 'beds': 0.006769194641117913,
 'security_deposit': 0.018782570504703354,
 'cleaning_fee': 0.023417515584165476,
 'guests_included': 0.007393921266038063,
 'extra_people': 0.01787880672986879,
 'minimum_nights': 0.013788798027244038,
 'maximum_nights': 0.015430949515226913,
 'availability_30': 0.021356013033860793,
 'availability_60': 0.010971383260116674,
 'availability_90': 0.012381931753733257,
 'availability_365': 0.017680737056601017,
 'number_of_reviews': 0.020349439432817733,
 'number_of_reviews_ltm': 0.01467605031083085,
 'calculated_host_listings_count': 0.014243026460991279,
 'calculated_host_listings_count_entire_homes': 0.013248170500019977,
 'calculated_host_listings_count_private_rooms': 0.003862472360577949,
 'calculated_host_listings_count_shared_rooms': 0.013360392799416075,
 'x': 0.023414422690070057,
 'y': 0.03054884

In [18]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17968 entries, 0 to 21311
Columns: 237 entries, host_is_superhost to suitable_for_events
dtypes: float32(3), int16(9), int32(1), int8(224)
memory usage: 4.6 MB


In [19]:
from xgboost import XGBRegressor as XGBR

from catboost import CatBoostRegressor as CTR

from lightgbm import LGBMRegressor as LGBMR

In [20]:
xgbr=XGBR().fit(X, y)
ctr=CTR(verbose=0).fit(X, y)
lgbmr=LGBMR().fit(X, y)

In [21]:
rfr.feature_importances_.sum()

1.0

In [22]:
dict(zip(X.columns, xgbr.feature_importances_))   

{'host_is_superhost': 0.0010294438,
 'accommodates': 0.0054049823,
 'bathrooms': 0.012421501,
 'bedrooms': 0.0226537,
 'beds': 0.0011232727,
 'security_deposit': 0.003688909,
 'cleaning_fee': 0.0031369466,
 'guests_included': 0.0026984473,
 'extra_people': 0.0027543698,
 'minimum_nights': 0.0024247218,
 'maximum_nights': 0.0018937322,
 'availability_30': 0.0040110266,
 'availability_60': 0.0018862492,
 'availability_90': 0.0025572907,
 'availability_365': 0.0015289497,
 'number_of_reviews': 0.0018239188,
 'number_of_reviews_ltm': 0.0023443415,
 'calculated_host_listings_count': 0.004720285,
 'calculated_host_listings_count_entire_homes': 0.0039821747,
 'calculated_host_listings_count_private_rooms': 0.0013931876,
 'calculated_host_listings_count_shared_rooms': 0.0312915,
 'x': 0.0012850672,
 'y': 0.002242861,
 'z': 0.0036633944,
 'property_type_apartment': 0.00079480914,
 'property_type_barn': 0.0,
 'property_type_bed_and_breakfast': 0.00043658746,
 'property_type_boutique_hotel': 0.00

In [23]:
dict(zip(X.columns, ctr.feature_importances_))   

{'host_is_superhost': 0.6524081803572713,
 'accommodates': 5.852880808944088,
 'bathrooms': 3.262348876606133,
 'bedrooms': 5.451325875826236,
 'beds': 0.6432610852984566,
 'security_deposit': 3.2977812826156567,
 'cleaning_fee': 4.723702465272869,
 'guests_included': 2.4054492915648207,
 'extra_people': 3.925814703712657,
 'minimum_nights': 2.4672951091173814,
 'maximum_nights': 1.7980534353943536,
 'availability_30': 1.8272394490019448,
 'availability_60': 1.4162180566852645,
 'availability_90': 1.7027400772184857,
 'availability_365': 1.8578182883483785,
 'number_of_reviews': 1.9610578544159647,
 'number_of_reviews_ltm': 1.9290601964645497,
 'calculated_host_listings_count': 2.5876806834376693,
 'calculated_host_listings_count_entire_homes': 3.9672865408535736,
 'calculated_host_listings_count_private_rooms': 0.6880719069286181,
 'calculated_host_listings_count_shared_rooms': 1.6174825702529967,
 'x': 1.7897962919787542,
 'y': 3.0275752384556416,
 'z': 4.883851898648892,
 'property_

In [24]:
dict(zip(X.columns, lgbmr.feature_importances_))  

{'host_is_superhost': 25,
 'accommodates': 125,
 'bathrooms': 17,
 'bedrooms': 71,
 'beds': 35,
 'security_deposit': 102,
 'cleaning_fee': 188,
 'guests_included': 78,
 'extra_people': 146,
 'minimum_nights': 136,
 'maximum_nights': 101,
 'availability_30': 76,
 'availability_60': 49,
 'availability_90': 66,
 'availability_365': 106,
 'number_of_reviews': 78,
 'number_of_reviews_ltm': 112,
 'calculated_host_listings_count': 162,
 'calculated_host_listings_count_entire_homes': 125,
 'calculated_host_listings_count_private_rooms': 43,
 'calculated_host_listings_count_shared_rooms': 28,
 'x': 113,
 'y': 156,
 'z': 176,
 'property_type_apartment': 2,
 'property_type_barn': 0,
 'property_type_bed_and_breakfast': 0,
 'property_type_boutique_hotel': 0,
 'property_type_bungalow': 0,
 'property_type_camper_rv': 0,
 'property_type_casa_particular_cuba': 0,
 'property_type_cave': 0,
 'property_type_chalet': 0,
 'property_type_condominium': 4,
 'property_type_dome_house': 0,
 'property_type_earth_